In [2]:
import pandas as pd
import numpy as np
import sklearn

In [10]:
small_data = pd.read_csv('small_data.csv')
small_data

,Sample,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
0,HP 2_8degC_1,2_8degC,1426172,1591380,100740,1972502,80250,3891814,2386481,513996,...,1582940,226467,2250788,856000,38063,364001,7301,5644816,23558,525597
1,HP 2_8degC_2,2_8degC,1517956,1686277,142948,2729996,85646,6069966,2952691,517222,...,1453751,279571,439349,205406,39952,134353,14407,6020618,23632,535842
2,HP 2_8degC_3,2_8degC,1664485,1827098,130359,2211164,94101,5831749,2782558,567592,...,1360706,251630,366872,158065,51457,195269,7742,6263993,24783,554371
3,HP 2_8degC_4,2_8degC,1159526,1294607,93469,1557122,63105,4042984,1781499,448708,...,699933,191838,224136,76907,39453,407070,5052,5277272,18253,480930
4,HP RT_1,RT,676603,762540,174855,3666034,96213,3692688,1314990,774759,...,2377553,239961,2110805,831730,50190,9796,1551,5116960,20178,428768
5,HP RT_2,RT,727034,807206,126424,2724566,66314,3069220,709370,698348,...,1849690,158963,512520,216781,46054,129625,1199,5653514,21400,316166
6,HP RT_3,RT,751274,832044,150530,4151215,87792,3886825,1073135,856665,...,1640040,230621,572257,232393,51065,50165,3331,6235203,25484,443281
7,HP RT_4,RT,593974,644762,149130,4112542,94976,4503385,658502,791352,...,1702734,261009,611152,255378,53199,6913,1454,6238185,28553,435282
8,HP RT_BHT_1,BHT,560734,625327,128295,2821701,88315,3315107,655856,579283,...,2555027,292232,405252,170659,43887,4526,113,6015531,34844,10153
9,HP RT_BHT_2,BHT,645725,690614,165197,2506349,90194,5603085,976099,546050,...,2743023,281899,478809,209904,53412,24346,2641,7386359,37677,13283


# Whether storage condition matter?

In [31]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

X = small_data.iloc[:,2:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]
pca = PCA(n_components=5)
pca.fit(X_centered)

X_PCA = pca.transform(X)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.42186834 0.20961367 0.10486136 0.05276885 0.04340681]
0.832519031135174
[10.24149086  7.21912418  5.10601924  3.6221259   3.28513817]


In [25]:
X = small_data.iloc[:,2:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]
pca = PCA(n_components=5)
pca.fit(X)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[0.66937651 0.10797663 0.06695484 0.04443856 0.036998  ]
0.9257445520262454


### explained variance by the first 5 PCs decreases if I centered the columns... why?

In [61]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X_centered)

print(loo)
y_pred = []
loo_decision_func = []
for train_index, test_index in loo.split(X_centered):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_centered[train_index], X_centered[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lsvc = LinearSVC(C = 0.05)
    lsvc.fit(X_train, y_train)
    print('true:',y_test)
    pred = lsvc.predict(X_test)
    y_pred.append(pred[0])
    print('predicted: ',pred)
    dec_func = lsvc.decision_function(X_test)
    print('decision function:',dec_func)
    loo_decision_func.append(dec_func[0])


LeaveOneOut()
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [0]
true: [0]
predicted:  [0]
decision function: [[-0.15019752 -0.68407323 -0.95223052 -0.22322374]]
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [1]
true: [0]
predicted:  [3]
decision function: [[-0.27984141 -1.07291517 -1.14675926  0.08698574]]
TRAIN: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [2]
true: [0]
predicted:  [0]
decision function: [[-0.08141998 -1.01553965 -0.95373016 -0.22504922]]
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14] TEST: [3]
true: [0]
predicted:  [3]
decision function: [[ 0.02840833 -0.60586815 -0.79836574  0.09781234]]
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14] TEST: [4]
true: [1]
predicted:  [1]
decision function: [[-0.75861084  0.39645883 -0.71035026 -1.08778119]]
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14] TEST: [5]
true: [1]
predicted:  [1]
decision function: [[-0.68260951  0.5109546  -0.41681156 -0.96388168]]
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 

In [68]:
loo_decision_func_df = pd.DataFrame(loo_decision_func)
loo_decision_func_df.to_csv('loo_decicion_function.csv')

In [52]:
print(y_pred)
print(y)

[0, 3, 0, 3, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0]
[0 0 0 0 1 1 1 1 2 2 2 3 3 3 3]


In [57]:
lsvc = LinearSVC(C = 0.05)
lsvc.fit(X_centered, y)
lsvc.decision_function(X_centered)

array([[ 0.35023824, -0.82537669, -0.97321099, -0.56514329],
       [ 0.39546938, -1.07293199, -1.14675059, -0.49596696],
       [ 0.34996235, -1.01557289, -0.97389152, -0.52873964],
       [ 0.38881872, -0.75713872, -0.87211414, -0.30703989],
       [-0.87776853,  0.68660718, -0.8491922 , -1.08779343],
       [-0.82782463,  0.73894019, -0.68886019, -0.98768765],
       [-0.95673418,  0.80231775, -0.90784458, -1.28094184],
       [-1.0290455 ,  0.68394527, -0.77587512, -1.11590289],
       [-0.93925979, -0.54730186,  0.64810768, -0.80048331],
       [-1.10313275, -0.81252337,  0.62773718, -0.89156572],
       [-0.83017705, -0.6814257 ,  0.55235533, -1.02364986],
       [-0.53002766, -0.84911148, -0.91481018,  0.46409772],
       [-0.62915081, -0.86722946, -0.83578801,  0.51990111],
       [-0.39798548, -0.9948801 , -0.98400036,  0.27917171],
       [-0.14533667, -1.15392955, -0.85881395,  0.29056065]])

In [56]:
decision_func = pd.DataFrame(lsvc.decision_function(X_centered))
decision_func.to_csv('decision_function_LinearSVC.csv')

# Feature analysis

In [3]:
small_data = pd.read_csv('small_data.csv')
small_data

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
0,0,1426172,1591380,100740,1972502,80250,3891814,2386481,513996,18467,...,1582940,226467,2250788,856000,38063,364001,7301,5644816,23558,525597
1,0,1517956,1686277,142948,2729996,85646,6069966,2952691,517222,23619,...,1453751,279571,439349,205406,39952,134353,14407,6020618,23632,535842
2,0,1664485,1827098,130359,2211164,94101,5831749,2782558,567592,17357,...,1360706,251630,366872,158065,51457,195269,7742,6263993,24783,554371
3,0,1159526,1294607,93469,1557122,63105,4042984,1781499,448708,15182,...,699933,191838,224136,76907,39453,407070,5052,5277272,18253,480930
4,1,676603,762540,174855,3666034,96213,3692688,1314990,774759,21773,...,2377553,239961,2110805,831730,50190,9796,1551,5116960,20178,428768
5,1,727034,807206,126424,2724566,66314,3069220,709370,698348,19513,...,1849690,158963,512520,216781,46054,129625,1199,5653514,21400,316166
6,1,751274,832044,150530,4151215,87792,3886825,1073135,856665,22056,...,1640040,230621,572257,232393,51065,50165,3331,6235203,25484,443281
7,1,593974,644762,149130,4112542,94976,4503385,658502,791352,25182,...,1702734,261009,611152,255378,53199,6913,1454,6238185,28553,435282
8,2,560734,625327,128295,2821701,88315,3315107,655856,579283,15528,...,2555027,292232,405252,170659,43887,4526,113,6015531,34844,10153
9,2,645725,690614,165197,2506349,90194,5603085,976099,546050,16290,...,2743023,281899,478809,209904,53412,24346,2641,7386359,37677,13283


In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

X = small_data.iloc[:,2:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]

Xctd_df = pd.DataFrame(X_centered)

In [38]:
# Xctd_df.iloc[14,184]
import numpy as np
from sklearn.linear_model import LinearRegression
score_lst = []
for c in range(185):
    reg = LinearRegression().fit(y.reshape(-1, 1), Xctd_df.iloc[:,c])
    score_lst.append(reg.score(y.reshape(-1, 1), Xctd_df.iloc[:,c]))


In [49]:
small_data.columns[1:]

Index(['Dimethylglycine_HMDB0000092', 'Glyoxylic acid_HMDB0000119',
       'Pyruvic acid_HMDB0000243', 'L-Lactic acid_HMDB0000190',
       'Alpha-Hydroxyisobutyric acid_HMDB0000729', 'Glycolic acid_HMDB0000115',
       'Caproic acid_HMDB0000535', 'L-Alanine_HMDB0000161',
       'Alpha-ketoisovaleric acid_HMDB0000019', 'Glycine_HMDB0000123',
       ...
       'Sucrose_HMDB0000258', 'MG(18:0e/0:0/0:0)_HMDB0011143',
       'Trehalose_HMDB0000975', 'D-Maltose_HMDB0000163',
       'Lactitol_HMDB0040937', 'Xanthylic acid_HMDB0001554',
       'Adenosine monophosphate_HMDB0000045', 'Cholesterol_HMDB0000067',
       'Lathosterol_HMDB0001170', '7b-Hydroxycholesterol_ HMDB0006119'],
      dtype='object', length=186)

In [50]:
sorted_lst = sorted(score_lst,reverse=True)
met_index = []
met_name_lst = []
for r in sorted_lst:
    indx = score_lst.index(r)
    met_index.append(indx)
    met_name_lst.append(small_data.columns[1:][indx])



In [52]:
metabolite_r2 = pd.DataFrame({'metabolite name':met_name_lst, 'R_square':sorted_lst})

In [53]:
metabolite_r2.to_excel('metabolite_r2.xlsx')

In [61]:
small_data.iloc[11:,:]

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
11,3,1080539,1201990,117525,1409613,71354,3202735,1332556,499903,15218,...,1993969,222420,8848515,3354293,53638,360786,8572,5729825,24820,390464
12,3,1399573,1557938,98968,1282860,66754,3560592,1625191,421395,17876,...,1860120,193325,786710,310727,44281,446259,4900,6538775,26001,410805
13,3,2075023,2308202,118428,1765898,92520,5017534,2137935,527216,21984,...,966079,265925,604507,242247,89800,128876,12029,6996995,29921,478722
14,3,1376262,1518899,133935,1568057,80930,5647582,2417288,433660,18470,...,962019,257435,304054,136614,42446,139153,10073,5841090,26089,413517


# Can one-way ANOVA answer the question?

In [65]:
from scipy.stats import f_oneway

f_lst = []
p_lst = []
for c in range(185):
    g0 = Xctd_df.iloc[0:4,c]
    g1 = Xctd_df.iloc[4:8,c]
    g2 = Xctd_df.iloc[8:11,c]
    g3 = Xctd_df.iloc[11:,c]
    f,p = f_oneway(g0, g1, g2, g3)
    f_lst.append(f)
    p_lst.append(p)


In [78]:
sorted_f = sorted(f_lst,reverse=True)
sorted_p = sorted(p_lst,reverse=False)
met_index = []
met_name_lst = []
for f in sorted_f:
    indx = f_lst.index(f)
    met_index.append(indx)
    met_name_lst.append(small_data.columns[1:][indx])

In [81]:
metabolite_f = pd.DataFrame({'metabolite name':met_name_lst, 'F_score':sorted_f,'P_value':sorted_p})
metabolite_f.to_excel('metabolite_f.xlsx')

In [86]:
metabolite_f.head(10)

,metabolite name,F_score,P_value
0,Mannose 6-phosphate_HMDB0001078,223.569290,3.894500e-10
1,Triethanolamine_HMDB0032538,190.517510,9.233940e-10
2,Lathosterol_HMDB0001170,100.416068,2.830777e-08
3,Phenyllactic acid_HMDB0000779,97.183556,3.365516e-08
4,Hypotaurine_HMDB0000965,94.727047,3.852847e-08
5,L-Aspartic acid_HMDB0000191,83.481473,7.494935e-08
6,Linoleic acid_HMDB0000673,78.384008,1.042886e-07
7,L-Glutamic acid_HMDB0000148,74.054558,1.403604e-07
8,"2,3-Dihydroxybutanedioic acid_HMDB0059916",69.073935,2.017531e-07
9,L-Arginine_HMDB0000517,68.651995,2.082858e-07


In [83]:
small_data.columns[1:][57:]

'Citramalic acid_HMDB0000426'

In [ ]:
fdfdasfdsafdsafdsaf